## Imports and reading in data:

In [1]:
import numpy as np
import pandas as pd
import altair as alt
import seaborn as sns

In [2]:
inflows = pd.read_parquet('C:/Users/bdion/Downloads/DSC180A/dsc180a/DSC180A/data/ucsd-inflows.pqt')
outflows = pd.read_parquet('C:/Users/bdion/Downloads/DSC180A/dsc180a/DSC180A/data/ucsd-outflows.pqt')

## Data Wrangling:

- Checking for missingness:

In [3]:
# inflows.isna().sum(), outflows.isna().sum(), inflows.isnull().sum(), outflows.isnull().sum()
# No NaNs

In [4]:
inflows.head()

,prism_consumer_id,prism_account_id,memo,amount,posted_date,category
0,0,acc_0,PAYCHECK,2477.02,2022-03-18,PAYCHECK
1,0,acc_0,EXTERNAL_TRANSFER,100.00,2022-10-25,EXTERNAL_TRANSFER
2,0,acc_0,MISCELLANEOUS,6.29,2022-08-26,MISCELLANEOUS
3,0,acc_0,EXTERNAL_TRANSFER,277.00,2022-06-03,EXTERNAL_TRANSFER
4,0,acc_0,EXTERNAL_TRANSFER,100.00,2022-07-29,EXTERNAL_TRANSFER


In [5]:
outflows.head()

,prism_consumer_id,prism_account_id,memo,amount,posted_date,category
0,0,acc_0,LOAN,900.60,2022-07-05,LOAN
1,0,acc_0,ATM_CASH,80.00,2022-03-25,ATM_CASH
2,0,acc_0,TST* Casa Del Rio - Exp Fairlawn OH 09/24,18.42,2022-09-26,FOOD_AND_BEVERAGES
3,0,acc_0,LOAN,634.00,2023-01-10,LOAN
4,0,acc_0,Buffalo Wild Wings,26.47,2022-09-12,FOOD_AND_BEVERAGES


    - No nulls in any datasets, but there are categories that are placeholders for NaNs since some transactions weren't categorized

- Making the posted_date column into datetime format:

In [6]:
inflows.posted_date  = pd.to_datetime(inflows.posted_date, format='%Y-%m-%d')
outflows.posted_date = pd.to_datetime(outflows.posted_date, format='%Y-%m-%d')

## EDA:

- Basic stat descriptions on the 'amount' column for both inflows/outflows

In [7]:
pd.concat([inflows[['amount']].describe(), outflows[['amount']].describe()], axis=1)

,amount,amount
count,5.131150e+05,2.597488e+06
mean,7.346998e+02,1.451264e+02
std,5.296566e+03,1.697875e+03
min,1.000000e-02,0.000000e+00
25%,2.058000e+01,9.630000e+00
50%,1.000000e+02,2.426000e+01
75%,5.000000e+02,6.636000e+01
max,1.154966e+06,6.548532e+05


    - Avg inflow transaction amount > avg outflow amount
    - Largest transaction is an inflow of $1.15 million
    - Funny that there's an outflow of $0, and an inflow of a penny lol
    - More outflows than there are inflows

- Checking for uniqueness among the columns:

In [8]:
print(inflows.prism_consumer_id.nunique(), outflows.prism_consumer_id.nunique())
print(inflows.prism_account_id.nunique() , outflows.prism_account_id.nunique())

2974 2968
4727 4622


    - consumer_id isn't unique to each account_id

- Number of inflows/outflows and total amount of money per account:

In [9]:
inflow_accs = inflows.groupby(['prism_account_id', 'prism_consumer_id']).agg({'amount':['count', 'sum']})
outflow_accs = outflows.groupby(['prism_account_id', 'prism_consumer_id']).agg({'amount':['count', 'sum']})

accs = pd.concat([inflow_accs, outflow_accs], axis=1)
accs

amount                                
                                    count         sum   count         sum
prism_account_id prism_consumer_id                                       
acc_0            0                   78.0   109811.91   644.0   111315.19
acc_1            0                   13.0      500.52     1.0     1000.38
acc_100          37                 423.0  9438368.57  1736.0  9364390.28
acc_1005         336                 97.0   154475.64   115.0   152694.69
acc_1006         336                 12.0       13.60     2.0     1990.00
...                                   ...         ...     ...         ...
acc_5255         1827                 NaN         NaN     1.0       19.99
acc_5257         1893                 NaN         NaN    30.0     2135.48
acc_5258         1893                 NaN         NaN     1.0       75.00
acc_5581         1956                 NaN         NaN     1.0        0.25
acc_9526         5943                 NaN         NaN     1.0        0.98

[4755 rows x 4 columns]

In [10]:
accs.sum()

amount  count    5.131150e+05
        sum      3.769855e+08
        count    2.597488e+06
        sum      3.769641e+08
dtype: float64

    - Total amount of money deposited is more than what is outflowing
        - $3.769855e+08 vs $3.769641e+08

In [11]:
len(np.intersect1d(inflow_accs.index, outflow_accs.index))

4594

In [12]:
len(np.union1d(inflow_accs.index, outflow_accs.index))

4755

In [13]:
len(np.intersect1d(inflow_accs.index, outflow_accs.index)) / len(np.union1d(inflow_accs.index, outflow_accs.index))

0.9661409043112513

    - Some accounts only have outflow/inflow, but both datasets have records of most accounts shared among them
    - There's a total of 4755 of unique accounts considering both inflow/outflow data sets, and 4594 are present in both of their records

In [14]:
accs.isna().sum()

amount  count     28
        sum       28
        count    133
        sum      133
dtype: int64

    - Specifically:
        - 133 only inflow ; 28 accounts only outflow

- Accounts with most transactions and most money moved:

In [15]:
inflow_accs.sort_values(('amount', 'count'), ascending=False)

amount           
                                    count        sum
prism_account_id prism_consumer_id                  
acc_7703         4120                1765  261668.46
acc_7806         4223                1298  109701.67
acc_6800         3217                1086   45771.38
acc_9332         5749                1019  283971.76
acc_9077         5494                1014  147917.88
...                                   ...        ...
acc_2713         954                    1     455.75
acc_552          190                    1      25.00
acc_551          190                    1      25.00
acc_550          190                    1      25.00
acc_3609         1287                   1     263.74

[4727 rows x 2 columns]

In [16]:
inflow_accs.sort_values(('amount', 'sum'), ascending=False)

amount            
                                    count         sum
prism_account_id prism_consumer_id                   
acc_100          37                   423  9438368.57
acc_4002         1433                 111  2839404.47
acc_3662         1308                 133  2242530.46
acc_8484         4901                  66  2226229.35
acc_1898         650                   83  2080324.48
...                                   ...         ...
acc_2554         885                    2        0.02
acc_4010         1438                   2        0.02
acc_653          217                    1        0.01
acc_4762         1719                   1        0.01
acc_4755         1719                   1        0.01

[4727 rows x 2 columns]

In [17]:
outflow_accs.sort_values(('amount', 'count'), ascending=False)

amount           
                                    count        sum
prism_account_id prism_consumer_id                  
acc_9475         5892                5470  286374.06
acc_7798         4215                5041  130622.67
acc_9421         5838                4822  112991.00
acc_8971         5388                4781  263498.55
acc_8339         4756                4460  140402.48
...                                   ...        ...
acc_1400         465                    1       2.00
acc_1395         475                    1    1500.00
acc_110          39                     1     176.71
acc_3109         1097                   1      84.96
acc_5258         1893                   1      75.00

[4622 rows x 2 columns]

In [18]:
outflow_accs.sort_values(('amount', 'sum'), ascending=False)

amount            
                                    count         sum
prism_account_id prism_consumer_id                   
acc_100          37                  1736  9364390.28
acc_4002         1433                 353  2685138.42
acc_8484         4901                 491  2170382.52
acc_1898         650                  244  2132970.75
acc_3662         1308                 569  1990302.52
...                                   ...         ...
acc_5137         1853                   1        0.00
acc_5138         1853                   1        0.00
acc_1770         527                    2        0.00
acc_1772         594                    5        0.00
acc_1399         403                   11        0.00

[4622 rows x 2 columns]

    - Accounts of interest: 
        - In terms of inflow:  acc_7703, acc_100
        - In terms of outflow: acc_9475, acc_100
    - acc_100 has largest inflow/outflow money

- Account transactions per category:

In [21]:
in_df = inflows.groupby(['category', 'prism_account_id']).agg({'amount':['count', 'sum'], 'memo':'unique'})

- Tracking the years of each inflow/outflow:

In [27]:
inflows['year']  = inflows.posted_date.dt.year
outflows['year'] = outflows.posted_date.dt.year
print(np.sort(inflows.year.unique()))
print(np.sort(outflows.year.unique()))

[2017 2018 2019 2020 2021 2022 2023]
[2017 2018 2019 2020 2021 2022 2023]


    - Both datasets have records from years 2017-2023

- Proportions of inflows/outflows in each observed year:

In [28]:
inflow_yrs = pd.DataFrame(inflows.year.value_counts())
outflow_yrs = pd.DataFrame(outflows.year.value_counts())
inflow_yrs['flow'] = 'inflow'
outflow_yrs['flow'] = 'outflow'
yrs = pd.concat([inflow_yrs, outflow_yrs], axis=0).reset_index().sort_values('year', ignore_index=True)

In [29]:
alt.Chart(yrs).mark_bar(opacity=0.3).encode(
    alt.X('year:Q', axis=alt.Axis(labelAngle=0, title="Year")),
    alt.Y('count', title='Count', stack=None),
    alt.Color("flow", legend=alt.Legend(title='Flow', symbolSize=300))
).properties(
    width=350, 
    height=500,
    title="Inflow/Outflow Proportions (2017-2023)"
)


alt.Chart(...)

    - Very few inflows and outflows in years 2017-2019 relative to the other years observed in the dataset
    - Most inflows and outflows are in 2022

- Amount of money moved/deposited throughout the years:

In [30]:
yrs_inflow  = inflows.groupby('year').agg({'amount':'sum'})
yrs_outflow = outflows.groupby('year').agg({'amount':'sum'})
yrs_inflow['flow'] = 'inflow'
yrs_outflow['flow'] = 'outflow'
yrs = pd.concat([yrs_inflow, yrs_outflow], axis=0).reset_index().sort_values('year', ignore_index=True)

alt.Chart(yrs).mark_line(strokeWidth=3).encode(
    alt.X('year:Q').axis(labelAngle=0).title("Year"),
    alt.Y('amount').title("Money"),
    alt.Color("flow", legend=alt.Legend(title='Flow', symbolSize=300)),
    tooltip=['flow', 'amount']
).properties(
    width=300,
    height=300,
    title="Money Throughout the Years",
).configure_legend(titleLimit=200, labelLimit=2000).interactive()

alt.Chart(...)

In [31]:
yrs

,year,amount,flow
0,2017,6.481660e+03,inflow
1,2017,2.400000e+01,outflow
2,2018,1.663071e+06,inflow
3,2018,1.529784e+06,outflow
4,2019,1.391400e+06,inflow
5,2019,1.669395e+06,outflow
6,2020,2.494055e+07,inflow
7,2020,2.457394e+07,outflow
8,2021,1.356071e+08,inflow
9,2021,1.342058e+08,outflow


    - Besides 2017, the total amount of money inflowing and outflowing is pretty close in number with an all-time high in 2022

-

In [32]:
inflows.groupby(['year', 'category']).agg({'amount':'count'})

amount
year category                     
2017 PAYCHECK_PLACEHOLDER        1
2018 DEPOSIT                   131
     EXTERNAL_TRANSFER          59
     INVESTMENT_INCOME         110
     MISCELLANEOUS              61
...                            ...
2023 REFUND                   2431
     SELF_TRANSFER           18297
     SMALL_DOLLAR_ADVANCE     3474
     TAX                       295
     UNEMPLOYMENT_BENEFITS      72

[78 rows x 1 columns]

-

In [33]:
outflows.groupby(['year', 'category']).agg({'amount':'count'})

amount
year category                   
2017 ACCOUNT_FEES              2
2018 ACCOUNT_FEES             46
     ATM_CASH                564
     AUTOMOTIVE              302
     CREDIT_CARD_PAYMENT     224
...                          ...
2023 RENT                    251
     SELF_TRANSFER         19957
     TAX                     237
     TRAVEL                 8614
     UNCATEGORIZED         12088

[160 rows x 1 columns]

In [34]:
cat = inflows.groupby('category').agg({'amount': ['count', 'sum']}).reset_index()
cnts, sums = cat[('amount', 'count')], cat[('amount', 'sum')]
cat.sort_values(('amount', 'count'), ignore_index=True)

category  amount             
                            count          sum
0               INSURANCE     174    338514.02
1   UNEMPLOYMENT_BENEFITS    1961   1610419.79
2                    LOAN    2513   4231710.73
3                     TAX    3405   5619601.78
4          OTHER_BENEFITS    7708  12175837.46
5    SMALL_DOLLAR_ADVANCE   13621   1906595.47
6       INVESTMENT_INCOME   17325   6887278.35
7                  REFUND   23220   5406440.27
8    PAYCHECK_PLACEHOLDER   26087  35706161.73
9                PAYCHECK   33138  54018520.29
10          MISCELLANEOUS   55648  52594312.59
11                DEPOSIT   61345  66602343.60
12          SELF_TRANSFER  110437  48301522.92
13      EXTERNAL_TRANSFER  156533  81586232.14

In [35]:
cat = outflows.groupby('category').agg({'amount': ['count', 'sum']}).reset_index()
cnts, sums = cat[('amount', 'count')], cat[('amount', 'sum')]
cat.sort_values(('amount', 'count'), ignore_index=True)

category  amount             
                          count          sum
0             AUTO_LOAN       1       200.00
1       BILLS_UTILITIES      23      1849.94
2      CHILD_DEPENDENTS     936     92310.31
3              PAYCHECK    1040   1303831.50
4              MORTGAGE    1119   1878837.62
5                  BNPL    1543     69737.49
6                  RENT    3147   2324934.30
7                   TAX    3186   4931463.39
8             OVERDRAFT    3386    232317.22
9             EDUCATION    4499    840825.25
10      GIFTS_DONATIONS    4719    642209.43
11     HOME_IMPROVEMENT    8600   1264462.16
12                 PETS    9266    668187.56
13            INSURANCE   30244   5314688.69
14         ACCOUNT_FEES   37511    625715.21
15   HEALTHCARE_MEDICAL   40842   2708153.04
16               TRAVEL   59647   3330811.10
17        SELF_TRANSFER   73281  16116654.62
18  CREDIT_CARD_PAYMENT   75506  42692941.23
19   ESSENTIAL_SERVICES   77137  11681316.66
20        ENTERTAINMENT   80885   2733419.26
21                 LOAN   90945  34315876.53
22        UNCATEGORIZED  117409  26805352.06
23             ATM_CASH  117651  54304232.38
24           AUTOMOTIVE  208579   9102938.62
25            GROCERIES  219331  11284707.29
26    EXTERNAL_TRANSFER  320998  91841663.61
27   FOOD_AND_BEVERAGES  481994  19552080.75
28  GENERAL_MERCHANDISE  524063  30302360.94

    - 'UNCATEGORIZED' is interesting. Is it in place for a NAN or is there just no appropriate category that described the outflow? 